# Read and clean a single file

In [1]:
import os
import sys
import logging
import pandas as pd

from pandas_profiling import ProfileReport


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulator import get_input_data

bucket = 'esp-big-data'
directory = 'BigData'

## Read Metadata
-----
General info for each relevant column

In [2]:
file = 'gs://{}/{}/data/metadatos-llamadas-urg-y-emer.csv'.format(bucket, directory)
df_metadata = pd.read_csv(file, encoding='latin1', sep=';')
df_metadata

,NOMBRE,DESCRIPCION
0,FECHA_INCIDENTE,Es la fecha el cual se registra la llamada del...
1,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,Es la fecha el cual se inicia el desplazamient...
2,CODIGO LOCALIDAD,Es el código de las 20 localidades de la ciuda...
3,LOCALIDAD,Es la localidad donde sucede el incidente.
4,EDAD,La edad del paciente.
5,UNIDAD,"La descripción de la edad si es en horas, días..."
6,GENERO,Es la distinción de genero del paciente
7,RED,Es la localización a nivel bogota de la red de...
8,TIPO_INCIDENTE,Es la descripción inicial que tipifica el cent...
9,PRIORIDAD,Es la tipificación según la prioridad del inci...


The final data must contains this and only this columns with that specific headers in order to standarize the data, this is a type of data dictionary but it also must include the data types (**schema**) of each field (**column**)

## Get raw data

In [98]:
raw_data = get_input_data(
    bucket            = bucket,
    initial_directory = directory,
    filename          = 'datos_abiertos_cons_diciembre_2019.csv'
)
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16243 entries, 0 to 16242
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NUMERO_INCIDENTE     16243 non-null  object
 1   FECHA_DESPACHO_518   16243 non-null  object
 2   CODIGO DE LOCALIDAD  16243 non-null  int64 
 3   LOCALIDAD            16243 non-null  object
 4   EDAD                 16243 non-null  int64 
 5   UNIDAD               16243 non-null  object
 6   GENERO               16243 non-null  object
 7   RED                  16243 non-null  object
 8   TIPO_INCIDENTE       16243 non-null  object
 9   PRIORIDAD            16243 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.2+ MB
None


It seems that this file contains already the fields, but need to rename them.

September file only contains 10 columns instead of 12, let's impute them

In [99]:
raw_data['MES'] = ['DICIEMBRE']*raw_data.shape[0]                    # fill the column with the same value
raw_data['CLASIFICACION_FINAL'] = ['SIN-DATO']*raw_data.shape[0]

### Rename Columns

In [100]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL',
        'TIPO DE INCIDIDENT'                : 'TIPO_INCIDENTE',                    # AGOSTO HAS THIS BAD COLUMN NAME
        'FECHA_DEPACHO_518'                 : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL', # Octubre and Diciembre has this bad column name
        
    },
    inplace=True
)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_DESPACHO_518,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,1178693196,2019-12-01 03:18:23,8,Kennedy,14,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA,DICIEMBRE,SIN-DATO
1,1176480196,2019-12-01 00:05:59,5,Usme,0,SIN/DATO,SIN/DATO,Sur,Accidente Tránsito con Heridos/Muertos,ALTA,DICIEMBRE,SIN-DATO
2,1180282196,2019-12-01 05:37:10,7,Bosa,32,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA,DICIEMBRE,SIN-DATO
3,1176520196,2019-12-01 00:07:43,3,Santa Fe,0,SIN/DATO,SIN/DATO,Norte,Lesiones Personales,ALTA,DICIEMBRE,SIN-DATO
4,1180873196,2019-12-01 07:14:08,5,Usme,53,Años,FEMENINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA,DICIEMBRE,SIN-DATO


### Check values per field
--------
1. LOCALIDAD
2. GENERO

In [101]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

 Kennedy              2317
Engativá              1644
Suba                  1471
Bosa                  1182
Ciudad Bolívar        1121
Puente Aranda          904
Fontibón               864
San Cristóbal          838
Rafael Uribe Uribe     772
Usaquén                770
Usme                   630
Tunjuelito             616
Santa Fe               589
Chapinero              556
Los Mártires           525
Barrios Unidos         513
Teusaquillo            497
Antonio Nariño         373
La Candelaria           60
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

It seems that in some fields exist a white space at the begining of the values, probably at the end too, lets clean them all

In [102]:
palabra = '   hola mundo    '
palabra.strip()

'hola mundo'

In [103]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2317
Engativá              1644
Suba                  1471
Bosa                  1182
Ciudad Bolívar        1121
Puente Aranda          904
Fontibón               864
San Cristóbal          838
Rafael Uribe Uribe     772
Usaquén                770
Usme                   630
Tunjuelito             616
Santa Fe               589
Chapinero              556
Los Mártires           525
Barrios Unidos         513
Teusaquillo            497
Antonio Nariño         373
La Candelaria           60
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [104]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [105]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_DESPACHO_518,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,1178693196,2019-12-01 03:18:23,8,Kennedy,14,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA,DICIEMBRE,SIN-DATO
1,1176480196,2019-12-01 00:05:59,5,Usme,0,SIN/DATO,SIN/DATO,Sur,Accidente Tránsito con Heridos/Muertos,ALTA,DICIEMBRE,SIN-DATO
2,1180282196,2019-12-01 05:37:10,7,Bosa,32,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA,DICIEMBRE,SIN-DATO
3,1176520196,2019-12-01 00:07:43,3,Santa Fe,0,SIN/DATO,SIN/DATO,Norte,Lesiones Personales,ALTA,DICIEMBRE,SIN-DATO
4,1180873196,2019-12-01 07:14:08,5,Usme,53,Años,FEMENINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA,DICIEMBRE,SIN-DATO


In [106]:
col = 'GENERO'
raw_data[col].value_counts(dropna=False)

SIN/DATO     8286
MASCULINO    4405
FEMENINO     3552
Name: GENERO, dtype: int64

In [107]:
col = 'GENERO'
raw_data[col].fillna('SIN-DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

SIN/DATO     8286
MASCULINO    4405
FEMENINO     3552
Name: GENERO, dtype: int64

In [108]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Heridos accidentales                      7390
Inconsciente / paro cardiorespiratorio    1743
Evento respiratorio                       1404
Convulsión                                1027
Enfermo                                    873
Trastorno mental                           650
Dolor torácico                             611
Intento de Suicidio                        543
Caída de Altura                            377
Patología ginecobstétrica                  371
Accidente Cerebro\nVascular                365
Síntomas gastrointestinales                275
Intoxicación                               153
S/D                                        138
Accidente Tránsito con Heridos/Muertos     108
Amenaza de\nSuicidio                        95
Violencia Sexual                            28
Sangrado Vaginal                            24
Lesiones Personales                         23
Quemaduras                                  16
Electrocución / Rescate                     10
Accidente Cer

In [109]:
col = 'CLASIFICACION_FINAL'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

SIN-DATO    16243
Name: CLASIFICACION_FINAL, dtype: int64

Here this columns contains a null value, aka NaN (this is numeric value that is Null), lets replace them first
* NaN --> SIN_DATO

In [110]:
raw_data[col].fillna('SIN-DATO').value_counts()

SIN-DATO    16243
Name: CLASIFICACION_FINAL, dtype: int64

In [111]:
col = 'CLASIFICACION_FINAL'
raw_data[col].fillna('SIN-DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

,NUMERO_INCIDENTE,FECHA_DESPACHO_518,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,1178693196,2019-12-01 03:18:23,8,Kennedy,14,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA,DICIEMBRE,SIN-DATO
1,1176480196,2019-12-01 00:05:59,5,Usme,0,SIN/DATO,SIN/DATO,Sur,Accidente Tránsito con Heridos/Muertos,ALTA,DICIEMBRE,SIN-DATO
2,1180282196,2019-12-01 05:37:10,7,Bosa,32,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA,DICIEMBRE,SIN-DATO
3,1176520196,2019-12-01 00:07:43,3,Santa Fe,0,SIN/DATO,SIN/DATO,Norte,Lesiones Personales,ALTA,DICIEMBRE,SIN-DATO
4,1180873196,2019-12-01 07:14:08,5,Usme,53,Años,FEMENINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA,DICIEMBRE,SIN-DATO


# Profile of the Data
------
first install pandas profiling, from a terminal

````
pip install pandas-profiling
````

In [112]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_diciembre_2019.html')

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Form the initial report of the file we see that are duplicates rows in the table, we need to subtract them

### Remove duplicates

In [113]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 16243
Number of rows after cleaning: 16162


### Save the final table
use to_csv to save the table

In [114]:
# saving in a local directory
raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_diciembre_2019.csv', encoding='latin1', sep=';', index=False)

# Next steps.... create the master table and analyze it